# Read a MySQL DataBase and generate specific output

In [1]:
from database_to_pandas import *
import sys

In [2]:
import pandas as pd

In [3]:
df=pd.read_json('data/db.json')

In [4]:
id=296

In [5]:
dfid=df[df['id']==id].index.values[0]

In [6]:
dfid

290

In [208]:
df.loc[dfid,'body']=r'''
%% Upper version
\def\Title{ }
\def\name{Kimy }
\def\lastname{Agudelo }
\def\ApplicationFor{Master degree at UNAM}
\def\years{two }
\def\program{program
%school
}
\def\possessive{her } %{his}
\def\Possessive{Her } %{His}
\def\gender{she } %{}
\def\Gender{She }
\def\pronoun{her } %{her}
\def\Pronoun{Her } %{Him}

\def\position{student}


%%Opening
\Title \name \lastname has asked me to write a letter of recommendation to accompany \possessive application for \ApplicationFor. 
I am very pleased to do so.

%% From when and how
I have known \name  since \gender was my student in the course of introductory particle physics two years ago.   \Gender is one of the best 
undergraduate students of our Institute. In particular, \gender was awarded a Youth Research Scholarship from our Institution to work 
in our Research Group in neutrino physics.

\name has just finished \pronoun bachelor thesis about Dirac neutrino masses 
in the context of gauged $B-L$ symmetries under my supervision. The final 
text was very well written and with very goog \LaTeX\ standars, and the
presentation was well organized and with outstanding clarity.
We are working in models that 
can explain the smallness of neutrino masses through the Dirac seesaw mechanism. 
During the development of the work, 
\gender has been able to solve all the problems and find creative solutions which allowed \pronoun to 
successfully implement the models, always solving the conceptual and technical difficulties by himself. 
I am very impressed with the results which are well beyond the expected level for an undergraduate student. 
In fact, we are now preparing a manuscript with the results to be submitted for publication in an international Journal in the field .

%, and now, when \Gender is one of my Master students, I am confident to give \pronoun any problem at the level of a doctorate student. We are now working in dark matter production during reheating.
%In fact, with the phenomenological analysis of the model in which \gender is actively participating, I believe that we are in a good track to have a manuscript suitable for publication in a few months. 

%%remark
\name is one of the most motivated students that I have ever had. 
\Gender imprints passion to everything  \gender does, 
and is self-sufficient and productive.
From the beginning I was impressed with \possessive ability to understand involved concepts about particle physics, 
which was demonstrated not only through the evaluation of the courses, 
but through his often thoughtful and penetrating questions.
 %\Gender is also very quick to carry out the proposed tasks.
\Gender has shown to be very reliable when performing several cumbersome calculations.
%,and to implement the results as programming codes.
%and very clear in showing the obtained results. 
In addition, \gender is self sufficient to establish the methodology of the proposed problems and to search 
for the relevant bibliography to solve the open issues.

%% Further comments
In my opinion \name has solid basis in several advanced topics \possessive has studied in the courses, 
and \gender is among the top five that I have taught.

%\name has proved to be a very quick studying, learning the advanced topics both in physics and programming and successfully combining both aspects. 
%Moreover, \gender is a good humored and friendly person with whom it is rather easy to establish any communication. 

%Add to all this that \name is a focused and determined young man. 

All together, those arguments lead me to emphasize my recommendation on \name{}'s candidature.
I am sure that \name can 
benefit a lot from \possessive participation in this 
\program, 
and because of that I endorse \possessive candidacy 
with confidence and enthusiasm, and very much hope that the committee judges his application favorably.

Please feel free to contact me for further information.
'''

In [209]:
df.loc[dfid,'preamble']=r'''
\usepackage{udea}
%\usepackage[spanish]{babel}
'''

In [210]:
df.to_json('data/db.json',orient='records',force_ascii=False)

In [211]:
import getpass
import pymysql
import unidecode
import pandas as pd
import tempfile
import subprocess
import sys
from latexfiles import *


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',500)

def generate_pdf(latexfile,
    pdf='output.pdf',
    dwld_dir='/home/restrepo/downloads',
    url_dir='file:///home/restrepo/downloads',id=None):
    """
    pdflatex upon latexfile to generate 'pdf'  file in 
    'dwld_dir' with browser link 'url_dir'
    """

    f=tempfile.NamedTemporaryFile('w',suffix='.tex',delete=False)
    f.write(latexfile)
    f.close()
    lo=subprocess.Popen('pdflatex {:s}'.format(f.name).split(),cwd='/tmp',
                        stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    pdffile='{:s}'.format(f.name).split('.tex')[0]+'.pdf'
    if lo[0].decode('utf8').find(pdffile.split('/')[-1])>-1:
        
        kk=subprocess.Popen('mv {:s} {:s}/{:s}'.format(pdffile,dwld_dir,pdf).split()).communicate()
        print('output in {:s}/{:s}'.format(url_dir,pdf))
        for suffix in  ['.aux','.log','.out','.tex']:
            sc=subprocess.Popen( ['rm', '{:s}'.format(f.name).split('.tex')[0]+suffix] ).communicate()
    else:
        print(lo[0].decode('utf8'))
        sys.exit('LaTeX ERROR:')    
    return latexfile

class database_to_pandas(object):
    """
    Convert Data Base  to pandas and filtered index to Series and 
    define methods to use them.
    Intialize with choose_db and main_table to build attibute: choose_db[main_table]
    See for example:
    {'Diego_Restrepo':'cartas','cartas_gfif':'cartas','seminarios':'talks'}
    The optional aux_tables must be set to '' if not required, otherwise 
    'cartas_sign' is used
    udea.sty and udea.pdf must be instaled
    """
    def __init__(self,host='localhost',user='root',choose_db='Diego_Restrepo',main_table='cartas',id=None,
                 aux_tables='cartas_sign',common_columns='signature',DB=True,verbose=False):
        if DB:
            password=getpass.getpass()
            self.choose_db={choose_db:main_table}
            self.DataBase=choose_db
            conn = pymysql.connect(host=host, port=3306, user=user, passwd=password, db=choose_db)
            df=pd.read_sql('select * from {};'.format(self.choose_db[choose_db]), con=conn)
        else:
            df=pd.read_json('data/db.json')
        #TODO: generilize to addtional tables
        if aux_tables:
            if DB:
                sg=pd.read_sql('select * from {};'.format(aux_tables), con=conn)
                conn.close()
            else:
                sg=pd.read_json('data/sg.json')

        self.pandas_DataFrame=df
        self.pandas_signature=sg
        #print(df.shape[0])
        #return df,sg
        if df.shape[0]:
            if not id:
                id=df.id.values[-1]
            self.pandas_Series=df[df.id==id].reset_index(drop=True).loc[0]
            if aux_tables:
                #TODO:
                self.pandas_Series['common_{}'.format(common_columns)]=sg[
                    sg[common_columns]==self.pandas_Series[common_columns]].reset_index(drop=True).loc[0,'sign']
            if verbose:
                print(self.pandas_Series)
        else:
            sys.exit('Error reading databases')
            return df

    
    def to_latex(self,func):
        '''Build a function to build the latex file with the self.Series keys
           and call it from here
        '''
        #TODO: check if class has methoed
        self.latexfile=func(self.pandas_Series)
        return self.latexfile
    #return generate_pdf(c)
    def to_pdf(self,func,**kwargs):
        '''Build a function to build the latex file with the self.Series keys
           and call it from here
        '''
        self.latexfile=func(self.pandas_Series)
        return generate_pdf(self.latexfile,**kwargs)
        
    def backup_database(self):
        '''
        Backup DB
        '''
    
        password=getpass.getpass()

        print('Trying to create backup file db_{:s}_backup.sql'.format( self.DataBase  )   )
        ok=subprocess.Popen('mysqldump -u root -p{:s} {:s} > db_{:s}_backup.sql'.format(
                         password,self.DataBase,self.DataBase),shell=True,
                         stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

        print()

        return ok
import getpass

In [212]:
data_base='Diego_Restrepo'
db=database_to_pandas(choose_db=data_base,main_table='cartas',id=id,DB=False)

In [213]:
latexfile=db.to_pdf(cartas,pdf='cartas.pdf',
            dwld_dir='/home/restrepo/downloads',
            url_dir='http://localhost/downloads',id=id)

output in http://localhost/downloads/cartas.pdf


In [13]:
id=None # last
#id=176 #Invitation letter
#id=281 # Invitado Nacional
#id=289 # Super recommendation corrected
#id=283 #certificado estancia
#id=307 # Generic Recommmendation letter
#id=306 #Generic Aval horas de investigación
#id=105
data_base='Diego_Restrepo' #Diego_Restrepo;cartas_gfif;seminarios
#data_base='cartas_gfif' #Diego_Restrepo;cartas_gfif;seminarios
try:
    if id:
        db=database_to_pandas(choose_db=data_base,main_table='cartas',id=id)
    else:    
        db=database_to_pandas(choose_db=data_base,main_table='cartas')
except KeyError:
    sys.exit('Uncomment Personal Data Base')       

 ·········


In [14]:
if id:
    latexfile=db.to_pdf(cartas,pdf='cartas.pdf',
            dwld_dir='/home/restrepo/downloads',
            url_dir='http://fisica.udea.edu.co/downloads',id=id)
else:
    latexfile=db.to_pdf(cartas,pdf='cartas.pdf',
            dwld_dir='/home/restrepo/downloads',
            url_dir='http://fisica.udea.edu.co/downloads')

NameError: name 'cartas' is not defined

### Backup database

In [14]:
db.pandas_DataFrame.to_json('data/db.json',orient='records',force_ascii=False)
db.pandas_signature.to_json('data/sg.json',orient='records',force_ascii=False)
db.backup_database()

········
Trying to create backup file db_Diego_Restrepo_backup.sql



(b'', b'')

In [10]:
!git commit -am 'fix signature json' > /dev/null
!git push origin master > /dev/null

[master ec5d327] fix signature json
 2 files changed, 71 insertions(+), 55 deletions(-)
ssh: connect to host github.com port 22: Connection timed out
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [22]:
db.pandas_DataFrame.loc[300]

id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             306
description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [26]:
print( db.pandas_DataFrame.loc[300,'para'] )

Señores\\
Consejo de Instituto\\
Instituto de Física\\
Universidad de Antioquia


In [25]:
print( db.pandas_DataFrame.loc[300,'body'] )

\def\proyecto{
%1: de movilidad Sao Paulo - Universidad de Antioquia  ``SPRINT – 3rd Edition 2019''
%2:``FAPESP - UNIVERSIDAD DE ANTIOQUIA 2019''
%3:
MÉTRICAS DE VINCULACIÓN UNIVERSIDAD ENTORNO ETAPA 3
}
\def\instituciones{
%1: el cual será presentado conjuntamente con investigadores de la Universidade Estadual de Campinas y el Instituto de Física Teórica de la  Universidade Estadual Paulista 
%2: el cual será presentado conjuntamente con investigadores de la Universidade Estadual de Campinas y el Instituto de Física Teórica de la  Universidade Estadual Paulista
%3
y del cual soy el investigador principal. Dicho proyecto se realiza en el marco del Acuerdo de Cooperación 01-2020, firmado entre la Vicerrectoría de Investigación, la Dirección de Relaciones Internacionales, la
Sede de Investigación Universitaria, el Programa de Gestión Tecnológica, la Facultad de Ciencias
Sociales y Humanas, la Facultad de Ciencias Exactas y Naturales, la Escuela Interamericana de
Bibliotecología y el Sist

\def\proyecto{
%1: de movilidad Sao Paulo - Universidad de Antioquia  ``SPRINT – 3rd Edition 2019''
%2:``FAPESP - UNIVERSIDAD DE ANTIOQUIA 2019''
%3: MÉTRICAS DE VINCULACIÓN UNIVERSIDAD ENTORNO ETAPA 3
%4:
``Desarrollo de algoritmos para la detección de neumonía y SARS-Cov-2 usando imágenes de Rayos-X.''
}

Solicito el aval para enviar el proyecto  \proyecto 
y del cual soy el investigador principal. Dicho proyecto se realiza en el marco del  
Fondo de apoyo a los grupos de trabajo de la iniciativa \#UdeA responde al Covid-19.
Dicho proyecto se hace en conjunto con la Facultad de Medicina y el CERN a través de un
convenio interinsitucional con la dedicación de 6 horas semanalas del coinvestigador del CERN: Omar Zapata.

La duración del proyecto será de 9 meses. 

Se adjunta el borrador de la propuesta en trámite y el presupuesto

### Other databases

In [ ]:
dbs=database_to_pandas(choose_db='seminarios',main_table='talks',aux_tables='')

In [10]:
lfs=dbs.to_pdf(seminarios,pdf='seminarios.pdf',
        dwld_dir='/home/restrepo/downloads',
        url_dir='http://gfif.udea.edu.co/downloads')

output in http://gfif.udea.edu.co/downloads/seminarios.pdf


### Get DataFrame

In [14]:
db.pandas_DataFrame.to_excel('kk.xlsx',index=False)

## Manual

In [9]:
latexfile

'\n%pago matrícula\n\\documentclass{letter}\n\\usepackage[utf8]{inputenc}\n\\usepackage{udea}\\usepackage[spanish]{babel}\n\n\\signature{\\includegraphics[scale=0.4]{firma}\\\\\r\n\\noindent\r\nDiego Restrepo\\\\\r\nProfesor de Tiempo Completo\\\\\r\nInstituto de Física\\\\\r\nUniversidad de Antioquia}\n\\begin{document}\n\\begin{letter}{Señores\\\\\r\nComité Científico SIU.\r\n\r\n\r\n}\n\\opening{Respetados señores:}\nSolicito el pago anticipado de las matrículas de maestría para los estudiantes\r\n\\begin{itemize}\r\n\\item María Coral Correa Tabares cc: 1.152.184.983, estudiante de la maestría en Ciencia, Tecnología e innovación,\r\n\\item Julian Andrés Calle Mosquera, C.C. 1.017.220.257, estudiante de la Maestría en Física,\r\n\\end{itemize}\r\n\r\n\r\ncon cargo a los recursos del proyecto en la convocatoria programática de CSH:\r\n\r\n 2016-13236 IMPACTO SOCIAL, CIENTÍFICO Y ACTIVIDADES DE VINCULACIÓN CON EL ENTORNO DE CENTROS DE INVESTIGACIÓN DE LA UNIVERSIDAD DE ANTIOQUIA,\r\n\

In [13]:
password=getpass.getpass()

········


In [14]:
choose_db=0
tipo_carta=['Diego_Restrepo', 'cartas_gfif']

conn = pymysql.connect(host='localhost', port=3306, user='root', passwd=password, db=tipo_carta[choose_db])

df=pd.read_sql('select * from cartas;', con=conn)

sg=pd.read_sql('select * from cartas_sign;', con=conn)

In [15]:
conn.close()Implicaciones de modelos de materia oscura en experimentos de detección directa

In [7]:
df=df[df.body.str.lower().str.contains('editor')]

In [16]:
df=df[df.id==281]

Last entry

In [17]:
df

,id,description,preamble,para,address,signature,opening,body,closing,fecha,link
275,281,,\usepackage{udea}\usepackage[spanish]{babel},Profesor Carlos Yaguna\\\r\nEscuela de Física\\\r\nUniversidad Pedagógica y \r\nTecnológica de Colombia\\\r\nUPTC,,Diego,None,"Tenemos el placer de invitarlo a una visita a nuestro grupo de Investigación en el Instituto de Física de la Universidad de Antioquia, del\r\n% 15 al 26 de octubre \r\n% 9 al 13 de octubre\r\n12 al 16 febrero \r\ndel presente año.\r\n\r\n Dentro del marco de esta visita esperamos continuar con nuestro trabajo sobre modelos con materia oscura y masas de neutrinos en el marco del proyecto de Colciencias \r\n``Implicaciones de modelos de materia oscura en experimentos de detección directa"" del ...",,2018-01-29,http://gfif.udea.edu.co/cartas/cartas.php?step=1&id=


In [18]:
c=df[df.id==df.id.values[-1]].reset_index(drop=True).loc[0]
c['sign']=sg[sg.signature==c.signature].reset_index(drop=True).loc[0,'sign']

In [19]:
c

id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             281
description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [20]:
row=c

In [21]:
lo=generate_pdf(row)

TypeError: must be str, not Series

In [22]:
print(lo[0].decode('utf8'))Implicaciones de modelos de materia oscura en experimentos de detección directa

NameError: name 'lo' is not defined

In [21]:
c.sign

"\\includegraphics[scale=0.4]{firma}\\\\\r\n\\noindent\r\nDiego A. Restrepo Quintero\\\\\r\nCoordinador\\\\\r\nGrupo de Fenomenología de Interacciones Fundamentales\\\\\r\nInstituto de F\\'\\i sica\\\\\r\nUniversidad de Antioquia"

In [18]:
ok

(b'', b'')

In [10]:
'mysqldump -u root -p{:s} {:s} > db_{:s}_backup.sql'.format(password,tipo_carta[NTC],tipo_carta[NTC]).split()

['mysqldump',
 '-u',
 'root',
 '-ppoioiulkj',
 'cartas_gfif',
 '>',
 'db_cartas_gfif_backup.sql']

## Search in body

In [50]:
id=99
if id >0:
    dfs=df[df.id==id].reset_index(drop=True)
else:    
    dfs=df[df.body.str.lower().map(unidecode.unidecode).str.contains('impacto social')].reset_index(drop=True)

In [ ]:
dfs

In [52]:
c=dfs.loc[dfs.index.values[-1]]

In [ ]:
c

In [ ]:
c.keys().values

In [ ]:
for k in c.keys():
    print(k)
    if c[k]==None:
        c[k]=''Implicaciones de modelos de materia oscura en experimentos de detección directa

In [52]:
c.closing

In [34]:
c.closing

'Best regards'

In [35]:
latexfile=r'''
%'''+c.description+r'''
\documentclass{letter}
\usepackage[utf8]{inputenc}
'''+c.preamble+r'''
'''+c.address+r'''
\signature{'''+sg[sg.signature==c.signature].sign.values[0]+r'''}
\begin{document}
\begin{letter}{'''+c.para+r'''}

\opening{'''+c.opening+r'''} 

'''+c.body+r'''

\closing{'''+c.closing+r'''}
\end{letter}
\end{document}
'''

TypeError: Can't convert 'NoneType' object to str implicitly

In [ ]:
print(latexfile)

In [58]:
f=tempfile.NamedTemporaryFile('w',suffix='.tex',delete=False)

In [59]:
f.write(latexfile)

2017

In [60]:
f.name

'/tmp/tmppt2pwnu1.tex'

In [61]:
f.close()

In [62]:
lo=subprocess.Popen('pdflatex {:s}'.format(f.name).split(),cwd='/tmp',stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

In [ ]:
pdffile='{:s}'.format(f.name).split('.tex')[0]+'.pdf'
if lo[0].decode('utf8').find(pdffile.split('/')[-1])>-1:
    pdf='carta_{:d}.pdf'.format(c['id'])
    dwld_dir='downloads'
    kk=subprocess.Popen('mv {:s} /var/www/{:s}/{:s}'.format(pdffile,dwld_dir,pdf).split()).communicate()
    print('output in http://gfif.udea.edu.co/{:s}/{:s}'.format(dwld_dir,pdf))
    for suffix in  ['.aux','.log','.out','.tex']:
        subprocess.Popen( ['rm', '{:s}'.format(f.name).split('.tex')[0]+suffix] ).communicate()
else:
    sys.exit('LaTeX ERROR:')    

In [14]:
%pycat database_to_pandas.py

In [16]:
r'https://scholar.google.com/scholar?hl=en&q=doi'

'https://scholar.google.com/scholar?hl=en&q=doi'